In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import os
import sys
import pickle
import matplotlib.pyplot as plt

In [2]:
from pandarallel import pandarallel
pandarallel.initialize()
import matplotlib.pyplot as plt
from sklearn.metrics import auc
try:
    import AD_result_fetcher
except:
    from . import AD_result_fetcher

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
# ===========================
# Try to see if we reveal 10 , what is the change in precision in the next 100 ?
# ===========================

In [4]:
DIR = 'us_import1'
PERCENTILE_THRESHOLD = 100
PERCENTILE_THRESHOLD = 100 - 5
REVEAL_STEP = 5
UPDATE_scoreUB_percentile =  10 
UPDATE_scoreUB = 0.5
id_col = 'PanjivaRecordID'
attribute_SHIPPER = 'ShipperPanjivaID'
attribute_CONSIGNEE = 'ConsigneePanjivaID'

In [5]:
labelled_results = AD_result_fetcher.read_in_AD_result(DIR)

score_threshold = np.percentile(labelled_results['score'],PERCENTILE_THRESHOLD)
print(score_threshold)

# The lowest score till which a new score should be calculated
UPDATE_scoreUB = np.percentile(labelled_results['score'],100-UPDATE_scoreUB_percentile)
print(UPDATE_scoreUB)

print('Number of positive anomalies : ', len(labelled_results.loc[labelled_results['label']==1]))
score_threshold = np.percentile(labelled_results['score'],PERCENTILE_THRESHOLD)

22648 22648
0.5650949312585849
0.504619568345812
Number of positive anomalies :  458


In [6]:
working_df = labelled_results.copy()
working_df['dynamic_score'] = labelled_results['score'].values
# ------------------
# Type conversion : to ensure no bugs
# ------------------

working_df['PanjivaRecordID'] = working_df['PanjivaRecordID'].astype(int)
working_df['ConsigneePanjivaID'] = working_df['ConsigneePanjivaID'].astype(int)
working_df['ShipperPanjivaID'] = working_df['ShipperPanjivaID'].astype(int)
working_df = working_df.reset_index(drop=True)

In [30]:
def eval_whiteList( target_df, ref_df ):
    global attribute_SHIPPER
    global attribute_CONSIGNEE
    global id_col
    global UPDATE_scoreUB
    orig_target_df = target_df.copy()
    fixed_df = target_df.loc[target_df['score']<UPDATE_scoreUB]
    target_df = target_df.loc[target_df['score']>=UPDATE_scoreUB]
    pos = ref_df.loc[ref_df['label']==1]
   
    target_entities = []
    target_entities.extend(pos[attribute_SHIPPER].values.tolist())
    target_entities.extend(pos[attribute_CONSIGNEE].values.tolist())
    # ----------------------
    # Do not update scores 
    # Simply order them by appending DFs
    # ----------------------
    df1 = target_df.loc[(target_df[attribute_CONSIGNEE].isin(target_entities)) & (target_df[attribute_CONSIGNEE].isin(target_entities))]
    remainder = target_df.loc[~(target_df[id_col].isin(df1[id_col].values.tolist()))]
    df2 = remainder.loc[(remainder[attribute_CONSIGNEE].isin(target_entities)) | (remainder[attribute_CONSIGNEE].isin(target_entities))]
    remainder = remainder.loc[~(remainder[id_col].isin(df2[id_col]))]
    res = df1.append(df2,ignore_index=True)
    res = res.append(remainder,ignore_index=True)
    res = res.append(fixed_df,ignore_index=True)
        
    for k in [50,100,150,200,250]:
        # without input?
        top_k_target = orig_target_df.head(k)
        wo_input_precison = len(top_k_target.loc[top_k_target['label']==1])/k
        
        top_k = res.head(k)
        prec = len(top_k.loc[top_k['label']==1])/k
        print(' precision at next top {} : {:.3f}'.format(k, prec), ' || Without input {:.3f}'.format(wo_input_precison) )

    
    return res


In [31]:
def eval_whiteBlackList( target_df, ref_df ):
    global attribute_SHIPPER
    global attribute_CONSIGNEE
    global id_col
    global UPDATE_scoreUB
    
    orig_target_df = target_df.copy()
    fixed_df = target_df.loc[target_df['score']<UPDATE_scoreUB]
    target_df = target_df.loc[target_df['score']>=UPDATE_scoreUB]
    pos = ref_df.loc[ref_df['label']==1]
    neg = ref_df.loc[ref_df['label']!=1]
    
    target_entities = []
    target_entities.extend(pos[attribute_SHIPPER].values.tolist())
    target_entities.extend(pos[attribute_CONSIGNEE].values.tolist())
    
    neg_target_entities = []
    neg_target_entities.extend(neg[attribute_SHIPPER].values.tolist())
    neg_target_entities.extend(neg[attribute_CONSIGNEE].values.tolist())
    # ----------------------
    # Do not update scores 
    # Simply order them by appending DFs
    # ----------------------
    df1 = target_df.loc[(target_df[attribute_CONSIGNEE].isin(target_entities)) & (target_df[attribute_CONSIGNEE].isin(target_entities))]
    remainder = target_df.loc[~(target_df[id_col].isin(df1[id_col].values.tolist()))]
    df2 = remainder.loc[(remainder[attribute_CONSIGNEE].isin(target_entities)) | (remainder[attribute_CONSIGNEE].isin(target_entities))]
    df2 = df2.loc[~(df2[attribute_CONSIGNEE].isin(neg_target_entities)) | ~(df2[attribute_CONSIGNEE].isin(neg_target_entities))]
    remainder = remainder.loc[~(remainder[id_col].isin(df2[id_col]))]
    df3 = remainder.loc[(remainder[attribute_CONSIGNEE].isin(neg_target_entities)) | (remainder[attribute_CONSIGNEE].isin(neg_target_entities))]
    remainder = remainder.loc[~(remainder[id_col].isin(df3[id_col]))]
    
    
    res = df1.append(df2,ignore_index=True)
    res = res.append(remainder,ignore_index=True)
    res = res.append(df3,ignore_index=True)
    res = res.append(fixed_df,ignore_index=True)
   
    for k in [50,100,150,200,250]:
        # without input?
        top_k_target = orig_target_df.head(k)
        wo_input_precison = len(top_k_target.loc[top_k_target['label']==1])/k
        top_k = res.head(k)
        prec = len(top_k.loc[top_k['label']==1])/k
        print(' precision at next top {} : {:.3f}'.format(k, prec), ' || Without input {:.3f}'.format(wo_input_precison) )

    
    return res


In [32]:
# -----------------------------
# label_top_k means how many positive anomalies get labelled by the annotator
# -----------------------------
def evaluate_on_input( df_cur, label_top_k = 25, scheme=1 ):
    global id_col
    df_cur = df_cur.sort_values(by='score',ascending=False)
    seen_ids = []
    count = 0
    labelled_df = None
    idx = 0
    for i,row in df_cur.iterrows():
        _id = int(row[id_col])
        idx = i
        seen_ids.append(_id)
        
        if row['label']==1: 
            count +=1
            if count > label_top_k : 
                break
        
    seen_ids = set(seen_ids)
    labelled_df = df_cur.iloc[:idx,:]
    unlabelled_df = df_cur.iloc[idx:,:]
    
    print('# of records revealed to find {} instances at the top :: {}'.format(
        label_top_k, 
        len(labelled_df))
    )
    # ----
    
    if scheme == 1 :
        eval_whiteList(unlabelled_df, labelled_df)
    elif scheme == 2 :
        eval_whiteBlackList(unlabelled_df, labelled_df)   
    return

    
    


In [33]:
evaluate_on_input( working_df.copy(), label_top_k = 5, scheme=1 )
evaluate_on_input( working_df.copy(), label_top_k = 10, scheme=1 )
evaluate_on_input( working_df.copy(), label_top_k = 20, scheme=1 )
evaluate_on_input( working_df.copy(), label_top_k = 30, scheme=1 )
evaluate_on_input( working_df.copy(), label_top_k = 40, scheme=1 )
evaluate_on_input( working_df.copy(), label_top_k = 50, scheme=1 )

# of records revealed to find 5 instances at the top :: 11
 precision at next top 50 : 0.520  || Without input 0.460
 precision at next top 100 : 0.460  || Without input 0.410
 precision at next top 150 : 0.513  || Without input 0.433
 precision at next top 200 : 0.475  || Without input 0.460
 precision at next top 250 : 0.476  || Without input 0.488
# of records revealed to find 10 instances at the top :: 24
 precision at next top 50 : 0.540  || Without input 0.420
 precision at next top 100 : 0.540  || Without input 0.410
 precision at next top 150 : 0.567  || Without input 0.433
 precision at next top 200 : 0.540  || Without input 0.480
 precision at next top 250 : 0.520  || Without input 0.488
# of records revealed to find 20 instances at the top :: 44
 precision at next top 50 : 0.600  || Without input 0.320
 precision at next top 100 : 0.600  || Without input 0.430
 precision at next top 150 : 0.627  || Without input 0.433
 precision at next top 200 : 0.665  || Without input 0.49

In [34]:
evaluate_on_input( working_df.copy(), label_top_k = 5, scheme=2 )
evaluate_on_input( working_df.copy(), label_top_k = 10, scheme=2 )
evaluate_on_input( working_df.copy(), label_top_k = 20, scheme=2 )
evaluate_on_input( working_df.copy(), label_top_k = 30, scheme=2 )
evaluate_on_input( working_df.copy(), label_top_k = 40, scheme=2 )
evaluate_on_input( working_df.copy(), label_top_k = 50, scheme=2 )

# of records revealed to find 5 instances at the top :: 11
 precision at next top 50 : 0.520  || Without input 0.460
 precision at next top 100 : 0.460  || Without input 0.410
 precision at next top 150 : 0.527  || Without input 0.433
 precision at next top 200 : 0.475  || Without input 0.460
 precision at next top 250 : 0.484  || Without input 0.488
# of records revealed to find 10 instances at the top :: 24
 precision at next top 50 : 0.540  || Without input 0.420
 precision at next top 100 : 0.540  || Without input 0.410
 precision at next top 150 : 0.567  || Without input 0.433
 precision at next top 200 : 0.540  || Without input 0.480
 precision at next top 250 : 0.524  || Without input 0.488
# of records revealed to find 20 instances at the top :: 44
 precision at next top 50 : 0.600  || Without input 0.320
 precision at next top 100 : 0.600  || Without input 0.430
 precision at next top 150 : 0.627  || Without input 0.433
 precision at next top 200 : 0.665  || Without input 0.49